In [1]:
import tensorflow as tf
import numpy as np
import emo
import numpy as np

Using TensorFlow backend.


In [2]:
#hyper-parameters
emb_dim=50
max_len=25
num_units=256
LR=0.005

In [3]:
# t. We use Cross Entropy
# with Softmax as our loss function [12], and Stochastic Gradient
# Descent (SGD) as our learner. We found the optimal batch size to
# be 4000 with a learning rate of 0.005

In [4]:
#placeholders
tf.reset_default_graph()
glove_in=tf.placeholder(dtype=tf.float32,shape=(None,max_len,emb_dim),name="glove_in")
sswe_in=tf.placeholder(dtype=tf.float32,shape=(None,max_len,emb_dim),name="sswe_in")
labels=tf.placeholder(dtype=tf.int32,shape=(None,1),name="labels")

In [5]:
with tf.variable_scope("glove_enc"):
    cell1=tf.nn.rnn_cell.LSTMCell(num_units)
    cell2=tf.nn.rnn_cell.LSTMCell(num_units)
    _,glove_states=tf.nn.bidirectional_dynamic_rnn(cell1,cell2,glove_in,dtype=tf.float32)    

In [6]:
with tf.variable_scope("sswe_enc"):
    cell1=tf.nn.rnn_cell.LSTMCell(num_units)
    cell2=tf.nn.rnn_cell.LSTMCell(num_units)
    _,sswe_states=tf.nn.bidirectional_dynamic_rnn(cell1,cell2,sswe_in,dtype=tf.float32)

In [7]:
with tf.variable_scope("merger"):
    features=tf.concat([glove_states[0].c,glove_states[1].c,glove_states[0].h,glove_states[1].h,
                        sswe_states[0].c,sswe_states[1].c,sswe_states[0].h,sswe_states[1].h]
                       ,axis=-1)

In [8]:
with tf.variable_scope("decoder"):
    dense1=tf.keras.layers.Dense(128,activation=tf.nn.leaky_relu)(features)
    dense2=tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(dense1)
    output=tf.keras.layers.Dense(4,activation="softmax",name="outputs")(dense2)
    

In [9]:
with tf.variable_scope("loss/opti"):
    loss=tf.keras.losses.sparse_categorical_crossentropy(labels,output)
    opti=tf.train.AdamOptimizer(learning_rate=LR,name='opti')
    step=opti.minimize(loss,name="step")

In [10]:
Sess=tf.Session()
Sess.run(tf.global_variables_initializer())

In [11]:
def one_epoch(batch_gen,i):
    m=0
    ep_l=0
    ep_acc=0
    for batch in batch_gen:
        m+=1
        if m==1:
            continue
        glov=batch[1][:,0]
        sswe=batch[1][:,1]
        labl=batch[2]
        labl=np.array(labl).reshape((-1,1))
        out,l,_=Sess.run([output,loss,step],feed_dict={glove_in:glov,sswe_in:sswe,labels:labl})
        l=l.sum()/l.shape[0]
        ep_l+=l
        acc=(out.argmax(-1)==labl[:,0]).sum()/labl.shape[0]
        ep_acc+=acc
        print("Epoch : ",i," batch : ",m," batch_loss :",l," batch_acc: ",acc)
    m-=1
    ep_l/=m
    ep_acc/=m
    print("Epoch : ",i,"epoch_loss : ",ep_l,"epoch_acc : ",ep_acc)

In [12]:
def fit(filename,mode,Glove,Sswe,batch_size,epochs,k=1):
    for i in range(epochs):
        y=emo.preprocessData(filename,mode,Glove,Sswe,batch_size,k)
        one_epoch(y,i)
        
        

In [13]:
Glove_emb=emo.loadEmbeddings(embeddingfile="./glove.6B.50d.txt",emb_dim=emb_dim)
# Sswe_emb=emo.loadEmbeddings(embeddingfile=,emb_dim=emb_dim)

In [14]:
zero_cat_fr=1
epochs=1
batch_size=4000
filename="./data/train.txt"
mode="train"

In [15]:
fit(filename,mode,Glove_emb,Glove_emb,batch_size,epochs,zero_cat_fr)

Epoch :  0  batch :  2  batch_loss : 1.38102734375  batch_acc:  0.18875
Epoch :  0  batch :  3  batch_loss : 5.40666357421875  batch_acc:  0.50125
Epoch :  0  batch :  4  batch_loss : 1.90913037109375  batch_acc:  0.1875
Epoch :  0  batch :  5  batch_loss : 1.40804443359375  batch_acc:  0.2035
Epoch :  0  batch :  6  batch_loss : 1.309914306640625  batch_acc:  0.4965
Epoch :  0  batch :  7  batch_loss : 1.25118017578125  batch_acc:  0.49025
Epoch :  0  batch :  8  batch_loss : 1.2189001012731482  batch_acc:  0.5115740740740741
Epoch :  0 epoch_loss :  1.9835514723358962 epoch_acc :  0.36847486772486765


In [26]:
def get_predicitons(test_data):
    glov=test_data[:,0]
    sswe=test_data[:,1]    
    out=Sess.run([output],feed_dict={glove_in:glov,sswe_in:sswe})
    return out

In [27]:
def get_metrics(pred,true,NUM_CLASSES):
    ground=[]
    true=np.array(true)
    for i in range(true.shape[0]):
        one=np.zeros((4,))
        one[true[i]]=1
        ground.append(one)
    ground=np.array(ground)
    
    emo.getMetrics(pred,ground,NUM_CLASSES)

In [28]:
indices,data,true=next(emo.preprocessData(filename,mode,Glove_emb,Glove_emb,4000,1))

In [29]:
pred=get_predicitons(data)[0]

In [30]:
(pred.argmax(-1)==true).sum()/true.shape

array([0.631])

In [31]:
get_metrics(pred,true,4)

True Positives per class :  [1742.  271.  190.  321.]
False Positives per class :  [1018.  244.   59.  155.]
False Negatives per class :  [246. 276. 523. 431.]
Class happy : Precision : 0.526, Recall : 0.495, F1 : 0.510
Class sad : Precision : 0.763, Recall : 0.266, F1 : 0.395
Class angry : Precision : 0.674, Recall : 0.427, F1 : 0.523
Ignoring the Others class, Macro Precision : 0.6545, Macro Recall : 0.3963, Macro F1 : 0.4937
Ignoring the Others class, Micro TP : 782, FP : 458, FN : 1230
Accuracy : 0.6310, Micro Precision : 0.6306, Micro Recall : 0.3887, Micro F1 : 0.4809


In [33]:
true,pred

(array([0, 3, 0, ..., 0, 0, 0]),
 array([[0.84576726, 0.00957406, 0.00755097, 0.13710772],
        [0.33892927, 0.06577966, 0.50501233, 0.09027879],
        [0.8723081 , 0.07595469, 0.01468266, 0.03705455],
        ...,
        [0.6370469 , 0.13873293, 0.03558363, 0.1886366 ],
        [0.80621886, 0.12938778, 0.00308445, 0.06130891],
        [0.6882101 , 0.01839887, 0.04380883, 0.24958214]], dtype=float32))

In [ ]:
x=np.array([1,2,3,4,5,6,7,8])
y=np.array([1,2,3,4,5,6,7,8])
np.random.seed(1)
np.random.shuffle(x)
x

In [ ]:
np.random.seed(1)
np.random.shuffle(y)
y

In [ ]:
pred.argmax(-1)[:200]

In [ ]:
true[:200]

In [ ]:
# #Testing
# test_path="./data/devwithoutlabels.txt"
# test_data=""
# test_data=next(emo.preprocessData(test_path,"test",Glove_emb,Glove_emb,batch_size=-1))
# # pred=get_predicitons(test_data[1])[0]

In [ ]:
# test_data[0][2754]

In [ ]:
# testData = pad_sequences(testSequences, maxlen=MAX_SEQUENCE_LENGTH)
# predictions = model.predict(testData, batch_size=BATCH_SIZE)
# predictions = predictions.argmax(axis=1)

# with io.open(solutionPath, "w", encoding="utf8") as fout:
#     fout.write('\t'.join(["id", "turn1", "turn2", "turn3", "label"]) + '\n')        
#     with io.open(testDataPath, encoding="utf8") as fin:
#         fin.readline()
#         for lineNum, line in enumerate(fin):
#             fout.write('\t'.join(line.strip().split('\t')[:4]) + '\t')
#             fout.write(label2emotion[predictions[lineNum]] + '\n')
# print("Completed. Model parameters: ")
# print("Learning rate : %.3f, LSTM Dim : %d, Dropout : %.3f, Batch_size : %d" 
#       % (LEARNING_RATE, LSTM_DIM, DROPOUT, BATCH_SIZE))

In [24]:
len(indices),data.shape

(2755, (2755, 2, 25, 50))

In [16]:
test_path="./data/devwithoutlabels.txt"
test_data=""
test_data=next(emo.preprocessData(test_path,"test",Glove_emb,Glove_emb,batch_size=2755))

In [17]:
test_data[1].shape

(2755, 2, 25, 50)